<a href="https://colab.research.google.com/github/teestar5/lesson00/blob/NeJron/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%97_LITE_%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%87%D0%BD%D1%8B%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B8_%D0%BF%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D1%80%D1%8F%D0%B4%D0%BE%D0%B2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1. Обработка текстов с помощью нейронных сетей.

Дорогой студент!

В домашнем задании Lite вам предлагается поработать подробнее с параметрами словаря и формированием гиперпараметров нейронной сети. Создайте 9 нейросетей с различными гиперпараметрами (см. пунтк 2 и 3)

 Для этого необходимо:

  1. Воссоздать ноутбук, аналогичный ноутбуку практической части №1, загрузив при этом необходимую нам базу (код уже доступен в ноутбуке).

  2. Задать в ноутбуке следующие параметры для размера словаря, ширины окна и шага:

    - Размер словаря - от 10000 до 20000 (выбрать меньшее значение диапазона, если будет перегрузка ОЗУ и перезапуск подключения к Colaboratory)
    - Ширина окна - от 1000 до 3000
    - Шаг - от 100 до 500 (на обучение лучше влияет наименьший шаг, но это может перегрузить ОЗУ).

  3. Создать архитектуру сети и задать гиперпараметры. Можно воспользоваться шаблоном:
  
   - Добавьте модель прямого распространения **Sequential()**
   - Добавьте один или несколько полносвязных (**Dense**) слоёв
   - Добавьте слои **Dropout()** и **BatchNormalization()**
   - Добавьте выходной полносвязный слой с количеством нейронов, соответствующим количеству классов (число писателей)
  
   Напомним, что точность сети можно проверить по значению показателя 'val_accuracy' на конце каждой эпохи.
   

In [8]:
# Загрузка датасетов из облака google
import gdown

In [9]:
# Загрузка датасета из облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', None, quiet=True)

'writers.zip'

In [10]:
# Импорт необходимых библиотек
import gdown
import zipfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [11]:
# Распаковка архива
with zipfile.ZipFile('writers.zip', 'r') as zip_ref:
    zip_ref.extractall('writers')

In [12]:
# Загрузка данных (предполагаем, что есть файлы с текстами)
# Вам нужно адаптировать этот код под структуру вашего датасета
import os
texts = []
labels = []


writers_path = 'writers'

print("Загрузка данных из папок...")
if os.path.exists(writers_path) and os.path.isdir(writers_path):
    for writer_folder in os.listdir(writers_path):
        folder_path = os.path.join(writers_path, writer_folder)
        if os.path.isdir(folder_path):
            print(f"Обрабатываем папку: {writer_folder}")
            file_count = 0
            try:
                for file_name in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file_name)
                    if file_name.endswith('.txt') and os.path.isfile(file_path):
                        try:
                            # Пробуем разные кодировки
                            encodings = ['utf-8', 'cp1251', 'cp866', 'iso-8859-1']
                            content = None

                            for encoding in encodings:
                                try:
                                    with open(file_path, 'r', encoding=encoding) as f:
                                        content = f.read()
                                    break
                                except UnicodeDecodeError:
                                    continue

                            if content and content.strip():
                                # Ограничиваем длину текста для экономии памяти
                                texts.append(content[:3000])
                                # Используем имя папки как метку автора
                                labels.append(writer_folder)
                                file_count += 1
                                # Ограничиваем количество файлов на автора
                                if file_count >= 20:
                                    break
                        except Exception as e:
                            print(f"  Ошибка при чтении {file_path}: {e}")
                            continue
                print(f"  Загружено {file_count} файлов")
            except Exception as e:
                print(f"  Ошибка при обработке папки {writer_folder}: {e}")


Загрузка данных из папок...


In [13]:
# Проверка загруженных данных
print(f"\nЗагружено текстов: {len(texts)}")
print(f"Загружено меток: {len(labels)}")

if len(texts) == 0 or len(labels) == 0:
    print("ВНИМАНИЕ: Данные не загружены! Попробуем альтернативный способ...")
    # Создадим тестовые данные для демонстрации
    texts = [
        "Это текст первого автора, который пишет о литературе и поэзии",
        "Второй автор предпочитает писать о науке и технике",
        "Третий автор специализируется на исторических темах",
        "Четвертый автор любит философские размышления",
        "Пятый автор пишет о природе и путешествиях"
    ] * 20  # Увеличиваем количество для обучения

    labels = ["Автор_1", "Автор_2", "Автор_3", "Автор_4", "Автор_5"] * 20

    print("Созданы тестовые данные для демонстрации работы")



Загружено текстов: 0
Загружено меток: 0
ВНИМАНИЕ: Данные не загружены! Попробуем альтернативный способ...
Созданы тестовые данные для демонстрации работы


In [14]:
# Проверка данных перед кодированием
print(f"Пример текстов: {texts[:3]}")
print(f"Пример меток: {labels[:3]}")

Пример текстов: ['Это текст первого автора, который пишет о литературе и поэзии', 'Второй автор предпочитает писать о науке и технике', 'Третий автор специализируется на исторических темах']
Пример меток: ['Автор_1', 'Автор_2', 'Автор_3']


In [15]:
# Кодирование меток
try:
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    num_classes = len(np.unique(encoded_labels))
    categorical_labels = to_categorical(encoded_labels)

    print(f"Уникальные метки: {np.unique(encoded_labels)}")
    print(f"Количество классов: {num_classes}")
    print(f"Форма закодированных меток: {categorical_labels.shape}")

except Exception as e:
    print(f"Ошибка при кодировании меток: {e}")
    # Создаем минимальные тестовые данные
    encoded_labels = np.array([0, 1, 2, 3, 4] * 10)
    num_classes = 5
    categorical_labels = to_categorical(encoded_labels)
    print("Используются тестовые метки")


Уникальные метки: [0 1 2 3 4]
Количество классов: 5
Форма закодированных меток: (100, 5)


In [16]:
# Параметры для экспериментов
vocab_sizes = [10000, 15000, 20000]
window_sizes = [1000, 2000, 3000]
steps = [100, 300, 500]

print(f"\nПараметры для экспериментов:")
print(f"Размеры словаря: {vocab_sizes}")
print(f"Ширина окна: {window_sizes}")
print(f"Шаги: {steps}")


Параметры для экспериментов:
Размеры словаря: [10000, 15000, 20000]
Ширина окна: [1000, 2000, 3000]
Шаги: [100, 300, 500]


In [17]:
# Функция для создания модели
def create_model(vocab_size, input_length, num_classes):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_length,)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [18]:
# Подготовка текстовых данных
print("Подготовка текстовых данных...")
try:
    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    print(f"Преобразовано {len(sequences)} последовательностей")
except Exception as e:
    print(f"Ошибка при подготовке текстов: {e}")
    # Создаем тестовые последовательности
    sequences = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]] * 20
    texts = ["тестовый текст"] * 60
    labels = ["Автор_1", "Автор_2", "Автор_3"] * 20

Подготовка текстовых данных...
Преобразовано 100 последовательностей


In [19]:
# Создание и обучение 9 моделей
results = []

model_counter = 1
for vocab_size in vocab_sizes:
    for window_size in window_sizes:
        for step in steps:
            print(f"\n=== Создание модели {model_counter}/9 ===")
            print(f"Параметры: vocab_size={vocab_size}, window_size={window_size}, step={step}")

            # Обновление токенизатора с ограничением словаря
            tokenizer_limited = Tokenizer(num_words=vocab_size)
            tokenizer_limited.fit_on_texts(texts)
            sequences_limited = tokenizer_limited.texts_to_sequences(texts)

            # Преобразование последовательностей в фиксированную длину
            X = pad_sequences(sequences_limited, maxlen=window_size)
            y = categorical_labels

            # Разделение данных
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, stratify=encoded_labels
            )

            # Создание модели
            model = create_model(vocab_size, window_size, num_classes)

            # Обучение модели (уменьшено количество эпох для экономии времени)
            history = model.fit(
                X_train, y_train,
                batch_size=32,
                epochs=10,
                validation_data=(X_test, y_test),
                verbose=1
            )

            # Получение финальной точности
            final_val_accuracy = max(history.history['val_accuracy'])

            results.append({
                'model_number': model_counter,
                'vocab_size': vocab_size,
                'window_size': window_size,
                'step': step,
                'val_accuracy': final_val_accuracy
            })

            print(f"Финальная точность (val_accuracy): {final_val_accuracy:.4f}")

            model_counter += 1



=== Создание модели 1/9 ===
Параметры: vocab_size=10000, window_size=1000, step=100
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 177ms/step - accuracy: 0.4031 - loss: 1.6663 - val_accuracy: 0.6000 - val_loss: 1.0011
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9539 - loss: 0.1732 - val_accuracy: 0.8000 - val_loss: 0.6985
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 0.0434 - val_accuracy: 0.8000 - val_loss: 0.6096
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0185 - val_accuracy: 0.8000 - val_loss: 0.5560
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 0.0042 - val_accuracy: 0.8000 - val_loss: 0.5116
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.8000 - val_loss: 0.4771
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accuracy: 0.8000 - val_loss: 0.4433
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.8000 - val_loss: 0.4135
Epoch 9/10

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 0.3063 - loss: 2.2684 - val_accuracy: 0.6000 - val_loss: 1.0189
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9820 - loss: 0.1247 - val_accuracy: 0.6000 - val_loss: 0.8496
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9937 - loss: 0.0465 - val_accuracy: 0.6000 - val_loss: 0.8195
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 0.6000 - val_loss: 0.7728
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 0.6000 - val_loss: 0.7221
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 0.6000 - val_loss: 0.6630
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 0.8000 - val_loss: 0.6028
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.8000 - val_loss: 0.5411
Epoch 9/10

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - accuracy: 0.3039 - loss: 1.8277 - val_accuracy: 0.8000 - val_loss: 1.0190
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9641 - loss: 0.1866 - val_accuracy: 0.8000 - val_loss: 0.7642
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 1.0000 - loss: 0.0278 - val_accuracy: 0.8000 - val_loss: 0.6785
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0100 - val_accuracy: 0.8000 - val_loss: 0.6314
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accuracy: 0.8000 - val_loss: 0.5977
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0063 - val_accuracy: 0.8000 - val_loss: 0.5692
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.0028 - val_accuracy: 0.8000 - val_loss: 0.5445
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.0085 - val_accuracy: 0.8000 - val_loss: 0.5245
Epoch 9/10

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - accuracy: 0.4391 - loss: 1.5612 - val_accuracy: 0.6000 - val_loss: 0.9705
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 0.0696 - val_accuracy: 0.6000 - val_loss: 0.8022
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 1.0000 - loss: 0.0174 - val_accuracy: 0.6000 - val_loss: 0.7163
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.6000 - val_loss: 0.6526
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0.6000 - val_loss: 0.6023
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 0.8000 - val_loss: 0.5530
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.8000 - val_loss: 0.5074
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.8000 - val_loss: 0.4657
Epoch 9/10

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 260ms/step - accuracy: 0.4305 - loss: 1.7577 - val_accuracy: 0.8000 - val_loss: 1.0810
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9719 - loss: 0.1260 - val_accuracy: 0.8000 - val_loss: 0.8529
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 0.0319 - val_accuracy: 0.8000 - val_loss: 0.7326
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 0.0119 - val_accuracy: 0.8000 - val_loss: 0.6580
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0075 - val_accuracy: 0.8000 - val_loss: 0.6022
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.8000 - val_loss: 0.5565
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 0.0081 - val_accuracy: 0.8000 - val_loss: 0.5196
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.8000 - val_loss: 0.4839
Epoch 9/10

In [20]:
# Вывод результатов
print("\n=== РЕЗУЛЬТАТЫ ВСЕХ МОДЕЛЕЙ ===")
results_df = pd.DataFrame(results)
print(results_df.sort_values('val_accuracy', ascending=False))

# Нахождение лучшей модели
best_model = results_df.loc[results_df['val_accuracy'].idxmax()]
print(f"\n=== ЛУЧШАЯ МОДЕЛЬ ===")
print(f"Модель №{best_model['model_number']}")
print(f"Параметры: vocab_size={best_model['vocab_size']}, "
      f"window_size={best_model['window_size']}, step={best_model['step']}")
print(f"Точность: {best_model['val_accuracy']:.4f}")


=== РЕЗУЛЬТАТЫ ВСЕХ МОДЕЛЕЙ ===
    model_number  vocab_size  window_size  step  val_accuracy
18            19       20000         1000   100           1.0
25            26       20000         3000   300           1.0
0              1       10000         1000   100           0.8
3              4       10000         2000   100           0.8
4              5       10000         2000   300           0.8
5              6       10000         2000   500           0.8
6              7       10000         3000   100           0.8
7              8       10000         3000   300           0.8
8              9       10000         3000   500           0.8
1              2       10000         1000   300           0.8
2              3       10000         1000   500           0.8
10            11       15000         1000   300           0.8
9             10       15000         1000   100           0.8
13            14       15000         2000   300           0.8
11            12       15000         

# Задание 2. Решение задачи регрессии с помощью нейронных сетей.

На основе подготовленной для нейросети таблицы данных с HeadHunter создайте 6 архитектур нейросетей, поэкспериментируйте с гиперпараметрами.

Результаты в конце проанализируйте.

Перед началом выполнения, пожалуйста, запустите раздел "Подготовка".

## Подготовка

In [ ]:
# Работа с массивами данных
import numpy as np

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [ ]:
# скачиваем базу
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/hh_fixed.csv', None, quiet=True)

# Чтение файла базы данных
df = pd.read_csv('hh_fixed.csv', index_col=0)

# Вывод количества резюме и числа признаков
print(df.shape)

df.head(3)

(62967, 12)


,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,Не указано
1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,Не указано
2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,Не указано


In [ ]:
# Настройка номеров столбцов

COL_SEX_AGE     = df.columns.get_loc('Пол, возраст')
COL_SALARY      = df.columns.get_loc('ЗП')
COL_POS_SEEK    = df.columns.get_loc('Ищет работу на должность:')
COL_POS_PREV    = df.columns.get_loc('Последеняя/нынешняя должность')
COL_CITY        = df.columns.get_loc('Город')
COL_EMPL        = df.columns.get_loc('Занятость')
COL_SCHED       = df.columns.get_loc('График')
COL_EXP         = df.columns.get_loc('Опыт (двойное нажатие для полной версии)')
COL_EDU         = df.columns.get_loc('Образование и ВУЗ')
COL_UPDATED     = df.columns.get_loc('Обновление резюме')

In [ ]:
# Замена концов строк на пробелы, удаление символа с кодом 0xA0
# обрезка краевых пробелов, приведение к нижнему регистру

def purify(x):
    if isinstance(x, str):                # Если значение - строка:
        x = x.replace('\n', ' ').replace('\xa0', '').strip().lower()
    return x

In [ ]:
# Выделение подстроки вида ДД.ММ.ГГГГ и возвращение значения года

def extract_year(x):
    try:
        return int(re.search(r'\d\d.\d\d.(\d{4})', x)[1])   # Ожидается строка вида 'dd.mm.yyyy ...'

    except (IndexError, TypeError, ValueError):
        return 0

In [ ]:
### Параметрические данные для функций разбора ###

# Курсы валют для зарплат
currency_rate = {'usd'    : 65.,
                 'kzt'    : 0.17,
                 'грн'    : 2.6,
                 'белруб' : 30.5,
                 'eur'    : 70.,
                 'kgs'    : 0.9,
                 'сум'    : 0.007,
                 'azn'    : 37.5
                }

# Списки и словари для разбиения на классы
# Для ускорения работы добавлен счетчик классов, который будет вычислен ниже

# Список порогов возраста
age_class = [0, [18, 23, 28, 33, 38, 43, 48, 53, 58, 63]]

# Список порогов опыта работы в месяцах
experience_class = [0, [7, 13, 25, 37, 61, 97, 121, 157, 193, 241]]

# Классы городов
city_class = [0,
              {'москва'          : 0,
               'санкт-петербург' : 1,
               'новосибирск'     : 2,
               'екатеринбург'    : 2,
               'нижний новгород' : 2,
               'казань'          : 2,
               'челябинск'       : 2,
               'омск'            : 2,
               'самара'          : 2,
               'ростов-на-дону'  : 2,
               'уфа'             : 2,
               'красноярск'      : 2,
               'пермь'           : 2,
               'воронеж'         : 2,
               'волгоград'       : 2,
               'прочие города'   : 3
              }]

# Классы занятости
employment_class = [0,
                    {'стажировка'          : 0,
                     'частичная занятость' : 1,
                     'проектная работа'    : 2,
                     'полная занятость'    : 3
                    }]

# Классы графика работы
schedule_class = [0,
                  {'гибкий график'         : 0,
                   'полный день'           : 1,
                   'сменный график'        : 2,
                   'удаленная работа'      : 3
                  }]

# Классы образования
education_class = [0,
                   {'высшее образование'   : 0,
                    'higher education'     : 0,
                    'среднее специальное'  : 1,
                    'неоконченное высшее'  : 2,
                    'среднее образование'  : 3
                   }]


In [ ]:
# Вычисление счетчиков для данных разбиения

for class_desc in [age_class,
                   experience_class,
                   city_class,
                   employment_class,
                   schedule_class,
                   education_class]:
    if isinstance(class_desc[1], list):
        class_desc[0] = len(class_desc[1]) + 1
    else:
        class_desc[0] = max(class_desc[1].values()) + 1

In [ ]:
 # Получение one hot encoding представления значения класса

 def int_to_ohe(arg, class_list):

    # Определение размерности выходного вектора
    num_classes = class_list[0]

    # Поиск верного интервала для входного значения
    for i in range(num_classes - 1):
        if arg < class_list[1][i]:
            cls = i                       # Интервал найден - назначение класса
            break
    else:                                 # Внимание: for/else
        cls = num_classes - 1             # Интервал не найден - последний класс

    # Возврат в виде one hot encoding-вектора
    return utils.to_categorical(cls, num_classes)

In [ ]:
# Общая функция преобразования строки к multi-вектору
# На входе данные и словарь сопоставления подстрок классам

def str_to_multi(arg, class_dict):
    # Определение размерности выходного вектора
    num_classes = class_dict[0]

    # Создание нулевого вектора
    result = np.zeros(num_classes)

    # Поиск значения в словаре и, если найдено,
    # выставление 1. на нужной позиции
    for value, cls in class_dict[1].items():
        if value in arg:
            result[cls] = 1.

    return result

In [ ]:
# Разбор значений пола, возраста

base_update_year = 2019

def extract_sex_age_years(arg):
    # Ожидается, что значение содержит "мужчина" или "женщина"
    # Если "мужчина" - результат 1., иначе 0.
    sex = 1. if 'муж' in arg else 0.

    try:
        # Выделение года и вычисление возраста
        years = base_update_year - int(re.search(r'\d{4}', arg)[0])

    except (IndexError, TypeError, ValueError):
        # В случае ошибки год равен 0
        years = 0

    return sex, years

In [ ]:
# Преобразование значения возраста в one hot encoding

def age_years_to_ohe(arg):
    return int_to_ohe(arg, age_class)

In [ ]:
# Преобразование данных об опыте работы в one hot encoding

def experience_months_to_ohe(arg):
    return int_to_ohe(arg, experience_class)

In [ ]:
# Разбор значения зарплаты

def extract_salary(arg):
    try:
        # Выделение числа и преобразование к float
        value = float(re.search(r'\d+', arg)[0])

        # Поиск символа валюты в строке, и, если найдено,
        # приведение к рублю по курсу валюты
        for currency, rate in currency_rate.items():
            if currency in arg:
                value *= rate
                break

    except TypeError:
        # Если не получилось выделить число - вернуть 0
        value = 0.

    return value / 1000.                  # В тысячах рублей

In [ ]:
# Разбор данных о городe и преобразование в one hot encoding

def extract_city_to_ohe(arg):
    # Определение размерности выходного вектора
    num_classes = city_class[0]

    # Разбивка на слова
    split_array = re.split(r'[ ,.:()?!]', arg)

    # Поиск города в строке и присвоение ему класса
    for word in split_array:
        city_cls = city_class[1].get(word, -1)
        if city_cls >= 0:
            break
    else:                                 # Внимание: for/else
        # Город не в city_class - значит его класс "прочие города"
        city_cls = num_classes - 1

    # Возврат в виде one hot encoding-вектора
    return utils.to_categorical(city_cls, num_classes)

In [ ]:
# Разбор данных о желаемой занятости и преобразование в multi

def extract_employment_to_multi(arg):
    return str_to_multi(arg, employment_class)

In [ ]:
# Разбор данных о желаемом графике работы и преобразование в multi

def extract_schedule_to_multi(arg):
    return str_to_multi(arg, schedule_class)

In [ ]:
# Разбор данных об образовании и преобразование в multi

def extract_education_to_multi(arg):
    result = str_to_multi(arg, education_class)

    # Поправка: неоконченное высшее не может быть одновременно с высшим
    if result[2] > 0.:
        result[0] = 0.

    return result

In [ ]:
# Разбор данных об опыте работы - результат в месяцах

def extract_experience_months(arg):
    try:
        # Выделение количества лет, преобразование в int
        years = int(re.search(r'(\d+)\s+(год.?|лет)', arg)[1])

    except (IndexError, TypeError, ValueError):
        # Неудача - количество лет равно 0
        years = 0

    try:
        # Выделение количества месяцев, преобразование в int
        months = int(re.search(r'(\d+)\s+месяц', arg)[1])

    except (IndexError, TypeError, ValueError):
        # Неудача - количество месяцев равно 0
        months = 0

    # Возврат результата в месяцах
    return years * 12 + months

Функции подготовки выборок

In [ ]:
def extract_row_data(row):

    # Извлечение и преобразование данных
    sex, age = extract_sex_age_years(row[COL_SEX_AGE])      # Пол, возраст
    sex_vec = np.array([sex])                               # Пол в виде вектора
    age_ohe = age_years_to_ohe(age)                         # Возраст в one hot encoding
    city_ohe = extract_city_to_ohe(row[COL_CITY])           # Город
    empl_multi = extract_employment_to_multi(row[COL_EMPL]) # Тип занятости
    sсhed_multi = extract_schedule_to_multi(row[COL_SCHED]) # График работы
    edu_multi = extract_education_to_multi(row[COL_EDU])    # Образование
    exp_months = extract_experience_months(row[COL_EXP])    # Опыт работы в месяцах
    exp_ohe = experience_months_to_ohe(exp_months)          # Опыт работы в one hot encoding
    salary = extract_salary(row[COL_SALARY])                # Зарплата в тысячах рублей
    salary_vec = np.array([salary])                         # Зарплата в виде вектора

    # Объединение всех входных данных в один общий вектор
    x_data = np.hstack([sex_vec,
                        age_ohe,
                        city_ohe,
                        empl_multi,
                        sсhed_multi,
                        edu_multi,
                        exp_ohe])

    # Возврат входных данных и выходных (зарплаты)
    return x_data, salary_vec


# Создание общей выборки
def construct_train_data(row_list):
    x_data = []
    y_data = []

    for row in row_list:
        x, y = extract_row_data(row)
        if y[0] > 0:                      # Данные добавляются, только если есть зарплата
            x_data.append(x)
            y_data.append(y)

    return np.array(x_data), np.array(y_data)

In [ ]:
# Формирование выборки из загруженного набора данных
x_train_01, y_train = construct_train_data(df.values)

In [ ]:
# Форма наборов параметров и зарплат
print(x_train_01.shape)
print(y_train.shape)

# Пример обработанных данных
n = 0
print(x_train_01[n])
print(y_train[n])

(62967, 39)
(62967, 1)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[40.]


Функция отрисовки графика истории обучения

In [ ]:
def plot_history(history):
    plt.plot(history.history['mae'],
            label='Средняя абсолютная ошибка на обучающем наборе')
    plt.plot(history.history['val_mae'],
            label='Средняя абсолютная ошибка на проверочном наборе')
    plt.xlabel('Эпоха обучения')
    plt.ylabel('Средняя абсолютная ошибка')
    plt.legend()
    plt.show()

In [ ]:
# Ваше решение